<a href="https://colab.research.google.com/github/JonHaz/Capstone-2-LVMH-Sales-in-Iowa/blob/main/LVMH_Capstone_Two_Pre_processing_and_Training_Data_Development.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
from sklearn.preprocessing import StandardScaler
from datetime import datetime,timedelta

In [2]:
# I am establishing a connection to my Sandbox within BigQuery so that I can run SQL queries to interact with the data here within Google Collab
from google.colab import auth
auth.authenticate_user()
print('Authenticated')


Authenticated


In [3]:
# Save output in a variable `df`
%%bigquery --project storied-sound-253213 df
SELECT *
FROM `bigquery-public-data.iowa_liquor_sales.sales`
WHERE vendor_name LIKE '%HENNESSY%'

In [4]:
df.dtypes

invoice_and_item_number     object
date                        object
store_number                object
store_name                  object
address                     object
city                        object
zip_code                    object
store_location              object
county_number               object
county                      object
category                    object
category_name               object
vendor_number               object
vendor_name                 object
item_number                 object
item_description            object
pack                         int64
bottle_volume_ml             int64
state_bottle_cost          float64
state_bottle_retail        float64
bottles_sold                 int64
sale_dollars               float64
volume_sold_liters         float64
volume_sold_gallons        float64
dtype: object

Step 1 : Create dummy or indicator features for categorical variables

In [5]:
df.nunique()

invoice_and_item_number    175379
date                         1256
store_number                 1952
store_name                   2027
address                      1898
city                          397
zip_code                      434
store_location               1678
county_number                  99
county                        129
category                       10
category_name                  10
vendor_number                   1
vendor_name                     1
item_number                   120
item_description              121
pack                            9
bottle_volume_ml               11
state_bottle_cost              86
state_bottle_retail            86
bottles_sold                   91
sale_dollars                 1112
volume_sold_liters            176
volume_sold_gallons           222
dtype: int64

So just by looking we can see that there are a ton of unique entries per column, since computing resources are finite I am going to cut down the number of features.

In [6]:
#applying a little data cleaning
#changing data types for 'date', 'pack', 'bottle_volume_ml'
df['date'] = pd.to_datetime(df['date'])
add_cat = ['pack', 'bottle_volume_ml']
df[add_cat] = df[add_cat].astype(str)
#removing fields that are unique identifiers, are redundant, or have more than 100 unique options
# ['invoice_and_item_number' , 'vendor_number','vendor_name', 'store_name', 'address','item_description', 'item_number', 'store_number', 'city', 'zip_code', 'store_location', 'county_number', 'county']
remove_cat = ['invoice_and_item_number' , 'vendor_number','vendor_name', 'store_name', 'address','item_description', 'item_number', 'store_number', 'city', 'zip_code', 'store_location', 'county_number', 'county']
df_cat = df.drop(columns=remove_cat)
df_cat.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 175379 entries, 0 to 175378
Data columns (total 11 columns):
 #   Column               Non-Null Count   Dtype         
---  ------               --------------   -----         
 0   date                 175379 non-null  datetime64[ns]
 1   category             174978 non-null  object        
 2   category_name        174978 non-null  object        
 3   pack                 175379 non-null  object        
 4   bottle_volume_ml     175379 non-null  object        
 5   state_bottle_cost    175379 non-null  float64       
 6   state_bottle_retail  175379 non-null  float64       
 7   bottles_sold         175379 non-null  int64         
 8   sale_dollars         175379 non-null  float64       
 9   volume_sold_liters   175379 non-null  float64       
 10  volume_sold_gallons  175379 non-null  float64       
dtypes: datetime64[ns](1), float64(5), int64(1), object(4)
memory usage: 14.7+ MB


First, we select all the columns that are categorical which are those with the data type = ‘object’, creating a data frame subset named ‘dfo’. Next, we concatenate the original data frame df while dropping those columns selected in the dfo, df.drop(dfo,axis=1), with the pandas.get_dummies(dfo) command, creating only indicator columns for the selected object data type columns and collating it with other numeric data frame columns.


In [7]:
# select object type columns
dfo=df_cat.select_dtypes(include=['object']) 
df = pd.concat([df.drop(dfo, axis=1), pd.get_dummies(dfo)], axis=1)

In [8]:
# lets move the field 'invoice_and_item_number' to the index
df = df.set_index('invoice_and_item_number')

# creating a series of dates
df_date = df['date']

# dropping columns so that the data can be scaled
cat = ['date','store_number',	'store_name',	'address',	'city',	'zip_code',	'store_location',	'county_number',	'county',	'vendor_number',	'vendor_name',	'item_number',	'item_description']
df_scaled = df.drop(columns=cat)
print(df_scaled.head())
print(df_date)

                         state_bottle_cost  ...  bottle_volume_ml_850
invoice_and_item_number                     ...                      
INV-30752900191                      34.99  ...                     0
INV-20918000010                      49.99  ...                     0
INV-10472100006                      49.99  ...                     0
INV-07182900001                      35.49  ...                     0
INV-06905200006                       5.99  ...                     0

[5 rows x 46 columns]
invoice_and_item_number
INV-30752900191   2020-10-05
INV-20918000010   2019-07-30
INV-10472100006   2018-02-19
INV-07182900001   2017-09-12
INV-06905200006   2017-08-28
                     ...    
INV-36711500140   2021-05-18
INV-35226700010   2021-03-23
INV-34178300020   2021-02-10
INV-37120400022   2021-06-02
INV-37409700059   2021-06-11
Name: date, Length: 175379, dtype: datetime64[ns]


Step 2. Standardize the magnitude of numeric features

In [26]:
#Using StandardScaler to normalize my data
scaler = StandardScaler()
scaler.fit(df_scaled)

StandardScaler(copy=True, with_mean=True, with_std=True)

Since I have time data I need to appropriately split it into training and testing data for an optimal model outcome.

In [27]:
#joining df_scaled to df_date on their indexes which is "invoice_and_item_number"
df = df_scaled.join(df_date, lsuffix='_df_scaled', rsuffix='_df_date')

#ordering the data by date
df = df.sort_values(by='date')


In [28]:
#establishing what my independent and dependent variable should be
X = df.loc[:,df.columns!="sale_dollars"]
y = df['sale_dollars']
test_size = 0.2
date_col = df.date

In [29]:
# using train_test_split making sure that my data is not shuffled since it is time series
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y,test_size=test_size, shuffle= False)